In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn
import os
import wandb
import yaml

In [3]:
#conf_file = "config-damage.yaml"
conf_file = "config-seg.yaml"

with open(conf_file) as f:
    conf_init = yaml.load(f)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [4]:
#os.environ['WANDB_MODE'] = 'dryrun'
wandb.init(project=conf_init['project'], config=conf_init)
conf = wandb.config

wandb: Wandb version 0.8.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [5]:
from pprint import pprint
pprint(dict(conf))

{'add_suppl': False,
 'amp_opt_level': 'O1',
 'aug_prob': 0.5,
 'batch_size': 1,
 'data_prefix': 'pre',
 'encoder': 'efficientnet-b7',
 'epochs': 100,
 'filter_none': False,
 'freeze_decoder_norm': True,
 'freeze_encoder_norm': True,
 'load_weights': '../weights/state_dict.pth',
 'loss_weights': {'focal': 4, 'jaccard': 1},
 'lr': 5e-05,
 'metric': 'building:f1',
 'mode': None,
 'nclasses': 1,
 'optim': 'adam',
 'project': 'sky-eye-full',
 'scheduler_factor': 0.5,
 'scheduler_patience': 5,
 'segmentation_arch': 'Linknet',
 'train_repeat': 1,
 'training_resolution': 1024,
 'training_scales': [0.75, 1.0]}


In [6]:
from glob import glob
from tqdm import tqdm_notebook as tqdm
import numpy as np
import torch
from torch import nn
from xv.util import vis_im_mask
import segmentation_models_pytorch as smp

In [7]:
train_dir = '../../datasets/xview/train'
test_dir = '../../datasets/xview/test'
suppl_dir = '../../datasets/xview/tier3'

In [8]:
import albumentations as al

augment = al.Compose([
        al.HorizontalFlip(p=conf.aug_prob),
        al.VerticalFlip(p=conf.aug_prob),
        al.RandomRotate90(p=conf.aug_prob),
        al.Transpose(p=conf.aug_prob),
        al.GridDistortion(p=conf.aug_prob, distort_limit=.2),
        al.ShiftScaleRotate(p=conf.aug_prob),
        al.RandomBrightnessContrast(p=conf.aug_prob),
])

In [9]:
from xv.nn.nets import DownscaleLayer, XVNet
import segmentation_models_pytorch as smp

segmentation_types = {
    'PSPNet': smp.PSPNet,
    'FPN': smp.FPN,
    'Linknet': smp.Linknet,
    'Unet': smp.Unet
}


model_classes = conf.nclasses
if conf.mode == "ordinal":
    model_classes -= 1

In [10]:
from xv.nn.layers import FrozenBatchNorm2d

model = segmentation_types[conf.segmentation_arch](conf.encoder,
                                                   classes=model_classes,
                                                   activation='sigmoid')
                                                   
if conf.load_weights:
    state_dict = torch.load(conf.load_weights)
    print(model.load_state_dict(state_dict))

preprocess_fn = get_preprocessing_fn(conf.encoder)

if conf.freeze_encoder_norm:
    model.encoder = FrozenBatchNorm2d.convert_frozen_batchnorm(model.encoder)
    
if conf.freeze_decoder_norm:
    model.decoder = FrozenBatchNorm2d.convert_frozen_batchnorm(model.decoder)
    
model = model.cuda()

<All keys matched successfully>


In [11]:
import random
from xv import dataset

random.seed(hash("😂"))


all_files = glob(f'{train_dir}/labels/*{conf.data_prefix}_disaster.json')
random.shuffle(all_files)

dev_ix = int(len(all_files)*.20)
dev_files = all_files[:dev_ix]
train_files = all_files[dev_ix:]

train_instances = dataset.get_instances(train_files, filter_none=conf.filter_none)

dev_instances = dataset.get_instances(dev_files, filter_none=conf.filter_none)

len(train_instances), len(dev_instances)

(2240, 559)

In [12]:
if conf.add_suppl:
    conf.train_repeat = 1
    train_instances *= conf.train_repeat
    suppl_files = glob(f'{suppl_dir}/labels/*{conf.data_prefix}_disaster.json')
    suppl_instances = dataset.get_instances(suppl_files, filter_none=conf.filter_none)
    train_instances += suppl_instances
    print(len(train_instances))

In [13]:
train_dataset = dataset.BuildingSegmentationDataset(
    instances=train_instances,
    nclasses=conf.nclasses,
    resolution=conf.training_resolution,
    augment=augment,
    preprocess_fn=preprocess_fn,
    mode=conf.mode,
)

dev_dataset = dataset.BuildingSegmentationDataset(
    instances=dev_instances,
    nclasses=conf.nclasses,
    resolution=conf.training_resolution,
    augment=None,
    preprocess_fn=preprocess_fn,
    mode=conf.mode,
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=conf.batch_size,
    shuffle=True,
    num_workers=10,
)

dev_loader = torch.utils.data.DataLoader(
    dev_dataset,
    batch_size=conf.batch_size,
    shuffle=False,
    num_workers=10,
)

In [14]:
from xv.nn.losses import loss_dict, WeightedLoss
from torch.nn.modules.loss import CrossEntropyLoss

if 'class_weight' in dict(conf):
    weights = torch.Tensor(conf.class_weight).float().cuda()
    loss = CrossEntropyLoss(weights)
else:
    loss = WeightedLoss({loss_dict[l]():w for l, w in conf.loss_weights.items()})

In [15]:
loss

WeightedLoss()

In [16]:
import apex

optims = {
    'adam': torch.optim.Adam
}

optim = optims[conf.optim](model.parameters(), lr=conf.lr)

In [17]:
from apex import amp
model, optim = amp.initialize(model, optim, opt_level=conf.amp_opt_level);

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [18]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim, factor=conf.scheduler_factor, patience=conf.scheduler_patience
)

In [19]:
from torchvision.ops import misc as misc_nn_ops
import random

class MultiScaleResize(nn.Module):
    def __init__(self, mode="categorical", scales = (0.5, 0.75, 1.)):
        super().__init__()
        self.mode = mode
        self.scales = scales
    
    @torch.no_grad()
    def forward(self, batch):
        scale = random.choice(self.scales)
        if scale == 1.:
            return batch
        if self.mode is None:
            im, mask = batch
            mask_dtype = mask.dtype
            im = torch.nn.functional.interpolate(im, scale_factor=scale, mode='bilinear', align_corners=False)
            mask = misc_nn_ops.interpolate(mask.float(), scale_factor=scale).to(mask_dtype)
            return im, mask
        if self.mode == "categorical":
            im, (damage_mask, damage) = batch
            dmg_msk_dtype = damage_mask.dtype()
            dmg_dtype = damage.dtype()
            im = torch.nn.functional.interpolate(im, scale_factor=scale, mode='bilinear', align_corners=False)
            damage_mask = misc_nn_ops.interpolate(damage_mask[None].float(), scale_factor=scale)[0].to(dmg_msk_dtype)
            damage_one_hot = torch.nn.functional.one_hot(damage).permute(0,3,1,2)
            damage = misc_nn_ops.interpolate(damage_one_hot.float(), scale_factor=scale).argmax(1)
            return im, (damage_mask, damage)

In [20]:
train_resize = MultiScaleResize(conf.mode, conf.training_scales)

In [21]:
best_score = 0
epoch = 0

In [26]:
torch.cuda.empty_cache()

In [ ]:
from xv import run

train_fn = run.train_segment if conf.nclasses == 1 else run.train_damage
eval_fn = run.evaluate_segment if conf.nclasses == 1 else run.evaluate_damage

for epoch in range(epoch, conf.epochs):
    metrics = {'epoch': epoch}
    train_metrics = train_fn(model, optim, train_loader, loss, train_resize=train_resize, mode=conf.mode)
    metrics.update(train_metrics)
    
    dev_metrics = eval_fn(model, dev_loader, loss, mode=conf.mode)
    metrics.update(dev_metrics)
    
    examples = run.sample_masks(model, dev_instances, preprocess_fn, n=3)
    metrics['examples'] = [wandb.Image(im, caption=f'mask:{ix}') for e in examples for ix, im in enumerate(e)]
    
    wandb.log(metrics)
    scheduler.step(metrics['loss'])
    score = metrics[conf.metric]

    if score > best_score:
        torch.save(model.state_dict(), os.path.join(wandb.run.dir, "state_dict.pth"))
        best_score = score

In [ ]:
dev_metrics = eval_fn(model, dev_loader, loss, mode=conf.mode)

In [23]:
dev_metrics

{'loss': tensor(0.0622, device='cuda:0'),
 'building:precision': tensor(0.8562, device='cuda:0'),
 'building:recall': tensor(0.8782, device='cuda:0'),
 'building:f1': tensor(0.8671, device='cuda:0')}

In [ ]:
ix = 1000
i = train_dataset[ix]
images, masks = i['images'], i['masks']
image = images['post']
image = np.array(train_dataset.inverse_transform_image(image))
util.vis_im_mask(image, masks['damage'], size=(512*2, 512*2), opacity=.3);

In [ ]:
from collections import Counter
counts = Counter(len(i['pre']['features']) for i in train_dataset.instances)

In [ ]:
for i in run.sample_masks(model, dev_instances, preprocess_fn, sz=1024):
    display(i)

In [ ]:
from PIL import Image
for i in random.sample(dev_instances, 1):
    with torch.no_grad():
        img = np.array(Image.open(i['file_name']))
        model_in = preprocess_fn(img).transpose(2,0,1)
        model_in = torch.tensor(model_in)
        model_in = model_in.reshape(1, *model_in.shape)
        mask = model(model_in.cuda())
        mask = np.array((mask > 0).cpu())

In [ ]:
from xv import run
tps, fps, fns = [], [], []
model = model.eval()
threshold=0.5
with torch.no_grad():
    for image, mask in tqdm(iter(dev_loader)):
        out = model(image.to('cuda'))
        for o, m in zip(out, mask):
            tp, fp, fn = run.get_tp_fp_fn(o, m, threshold)
            tps.append(np.array(torch.tensor(tp).cpu()))
            fps.append(np.array(torch.tensor(fp).cpu()))
            fns.append(np.array(torch.tensor(fn).cpu()))
            
import pandas as pd
df = pd.DataFrame({'tp': tps, 'fp': fps, 'fn':fns})

In [ ]:
idx = (df.fp + df.fn).sort_values(ascending=False).index
df.iloc[idx].head()

from PIL import Image
from xv import util
import cv2


i = dev_instances[ix]
sz = conf.training_resolution

#def analyse_instance(i):
with torch.no_grad():
    img = np.array(Image.open(i['file_name']))
    img = cv2.resize(img, (sz,sz))
    model_in = preprocess_fn(img).transpose(2,0,1)
    model_in = torch.tensor(model_in).float()
    model_in = model_in.reshape(1, *model_in.shape)
    mask = model(model_in.cuda())
    mask = np.array((mask > 0).cpu())
    im = util.vis_im_mask(img, mask[0], opacity=.3, size=(sz,sz))
    
_, target = dev_dataset[ix]
mask = mask.astype(bool).reshape(sz,sz)
target = target.astype(bool).reshape(sz,sz)

tp = target & mask
fp = mask & ~target
fn = target & ~mask

util.vis_im_mask(img, target, opacity=.2, colours=("blue", "yellow", "red"))
util.vis_im_mask(img, np.stack((tp, fn, fp)), opacity=.2, colours=("blue", "yellow", "red"))

In [ ]:
tta_model = tta.SegmentationTTAWrapper(model, transforms, merge_mode='mean')